# Colour Histogram

In [ ]:
import cv2
import matplotlib.pyplot as plt

def plot_color_histogram(image_paths):
    for image in image_paths:
        image_bgr = cv2.imread(image)
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        colors = ('r', 'g', 'b')
        for i, color in enumerate(colors):
            hist = cv2.calcHist([image_rgb], [i], None, [256], [0, 256])
            plt.plot(hist, color=color)

        plt.title('Histogram for Red, Green, and Blue channels')
        plt.xlabel('Pixel Intensity')
        plt.ylabel('Frequency')
        plt.xlim([0, 256])
        plt.show()

image_paths = [
    r'Dataset\training\Bleeding\images\04a78ef00c5245e0_11223_1.jpg',
    r'Dataset\training\Worms\images\aug__0_28.jpg'
]
plot_color_histogram(image_paths)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_color_histogram(image, bins=256):
    color = ('b', 'g', 'r')
    for i, col in enumerate(color):
        hist = cv2.calcHist([image], [i], None, [bins], [0, 256])
        plt.plot(hist, color=col)
        plt.xlim([0, 256])
    plt.title('Histogram for Blue, Green, and Red channels')
    plt.show()

image = cv2.imread(r'blue.png')
plot_color_histogram(image)


# Wavelet Transform

In [ ]:
import cv2
import numpy as np
import pywt
import matplotlib.pyplot as plt

def wavelet_decomposition_and_feature_extraction(img, wavelet='db2', level=2):
    features = []
    channels = cv2.split(img)

    for channel in channels:
        coeffs = pywt.wavedec2(channel, wavelet, level=level)
        for coeff in coeffs:
            if isinstance(coeff, tuple):
                for detail in coeff:
                    features.append(np.mean(detail))
                    features.append(np.std(detail))
                    features.append(np.median(detail))
                    features.append(np.sum(np.abs(detail)))
            else:
                features.append(np.mean(coeff))
                features.append(np.std(coeff))
                features.append(np.median(coeff))
                features.append(np.sum(np.abs(coeff)))
    
    return np.array(features)

def plot_wavelet_decomposition(img, wavelet='db2'):
    channels = cv2.split(img)
    titles = ['Red Channel', 'Green Channel', 'Blue Channel']
    
    plt.figure(figsize=(12, 8))
    
    for i, channel in enumerate(channels):
        coeffs = pywt.dwt2(channel, wavelet)
        LL, (LH, HL, HH) = coeffs

        plt.subplot(3, 4, i*4+1)
        plt.imshow(LL, cmap='gray')
        plt.title(f'{titles[i]} Approximation (LL)')
        plt.axis('off')

        plt.subplot(3, 4, i*4+2)
        plt.imshow(LH, cmap='gray')
        plt.title(f'{titles[i]} Horizontal detail (LH)')
        plt.axis('off')

        plt.subplot(3, 4, i*4+3)
        plt.imshow(HL, cmap='gray')
        plt.title(f'{titles[i]} Vertical detail (HL)')
        plt.axis('off')

        plt.subplot(3, 4, i*4+4)
        plt.imshow(HH, cmap='gray')
        plt.title(f'{titles[i]} Diagonal detail (HH)')
        plt.axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    img = cv2.imread(r'Dataset\training\Bleeding\images\04a78ef00c5245e0_11213_1.jpg')
    img = cv2.resize(img, (224, 224))
    wavelet_type = 'db2'
    features = wavelet_decomposition_and_feature_extraction(img, wavelet=wavelet_type, level=2)
    print("Extracted Wavelet Features: ", features)
    plot_wavelet_decomposition(img, wavelet=wavelet_type)


# Fourier Transform

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def fft_feature_extraction(img):
    features = []
    channels = cv2.split(img)
    
    for channel in channels:
        f = np.fft.fft2(channel)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
        phase_spectrum = np.angle(fshift)
        
        features.append(np.mean(magnitude_spectrum))
        features.append(np.std(magnitude_spectrum))
        features.append(np.median(magnitude_spectrum))
        features.append(np.sum(np.abs(magnitude_spectrum)))
        
        features.append(np.mean(phase_spectrum))
        features.append(np.std(phase_spectrum))
        features.append(np.median(phase_spectrum))
        features.append(np.sum(np.abs(phase_spectrum)))
        
    return np.array(features)

def plot_fft_magnitude(img):
    channels = cv2.split(img)
    titles = ['Red Channel', 'Green Channel', 'Blue Channel']
    
    plt.figure(figsize=(12, 8))

    for i, channel in enumerate(channels):
        f = np.fft.fft2(channel)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1)
        
        plt.subplot(1, 3, i + 1)
        plt.imshow(magnitude_spectrum, cmap='gray')
        plt.title(f'{titles[i]} Magnitude Spectrum')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    img = cv2.imread(r'Dataset\training\Bleeding\images\04a78ef00c5245e0_11213_1.jpg')
    img = cv2.resize(img, (224, 224))
    features = fft_feature_extraction(img)
    print("Extracted FFT Features: ", len(features))
    plot_fft_magnitude(img)
